In [59]:
import pandas as pd
import numpy as np

inputs = pd.read_csv('inputs.csv', delimiter=';', index_col=0)
outputs = pd.read_csv('outputs.csv', delimiter=';', index_col=0)

airports = pd.concat([inputs, outputs], axis=1)

airports.head()

,i1,i2,i3,i4,o1,o2
WAW,10.5,36,129.4,7.0,9.5,129.7
KRK,3.1,19,31.6,7.9,2.9,31.3
KAT,3.6,32,57.4,10.5,2.4,21.1
WRO,1.5,12,18.0,3.0,1.5,18.8
POZ,1.5,10,24.0,4.0,1.3,16.2


Efectivness

In [60]:
from pulp import *

es = []

for index1, row1 in airports.iterrows():
    prob = LpProblem("Airports", LpMaximize)
    u1 = LpVariable("u1", lowBound=0)
    u2 = LpVariable("u2", lowBound=0)
    v1 = LpVariable("v1", lowBound=0)
    v2 = LpVariable("v2", lowBound=0)
    v3 = LpVariable("v3", lowBound=0)
    v4 = LpVariable("v4", lowBound=0)

    prob += u1 * row1['o1'] + u2 * row1['o2'], "Objective function"

    prob += v1*row1['i1'] + v2*row1['i2'] + v3*row1['i3'] + v4*row1['i4'] == 1

    for index2, row2 in airports.iterrows():
        prob += u1*row2['o1'] + u2*row2['o2'] <= v1*row2['i1'] + v2*row2['i2'] + v3*row2['i3'] + v4*row2['i4']

    prob.solve()

    e = (u1.varValue * row1['o1'] + u2.varValue * row1['o2']) / (v1.varValue * row1['i1'] + v2.varValue * row1['i2'] + v3.varValue * row1['i3'] + v4.varValue * row1['i4'])

    e = e.round(4)

    es.append((index1, e))

es_dict = dict(es)

airports['e'] = airports.index.map(es_dict)

airports.head()

,i1,i2,i3,i4,o1,o2,e
WAW,10.5,36,129.4,7.0,9.5,129.7,1.0000
KRK,3.1,19,31.6,7.9,2.9,31.3,1.0000
KAT,3.6,32,57.4,10.5,2.4,21.1,0.5912
WRO,1.5,12,18.0,3.0,1.5,18.8,1.0000
POZ,1.5,10,24.0,4.0,1.3,16.2,0.7998


Adjustments

In [ ]:
airports_adjusted = airports.copy()

for index, row in airports.iterrows():
    if row['e'] < 1:
        airports_adjusted.loc[index, 'o1'] = row['o1'] * row['e']
        airports_adjusted.loc[index, 'o2'] = row['o2'] * row['e']